In [1]:
import torch
import numpy as np
import torchsummary
import torchvision

In [79]:
class DenseNet(torch.nn.Module):
    def __init__(self, growth_rate=12, num_layers=100, theta=0.5):
        super().__init__()

        num_bottle_layers = (num_layers-4) // 6
        self.conv = torch.nn.Conv2d(3, growth_rate*2, kernel_size=3, stride=1, padding=1, bias=True)

        self.dense_1 = DenseBlock(in_channels=growth_rate*2, num_layers=num_bottle_layers, growth_rate=growth_rate)
        in_trans_1 = growth_rate*2 + growth_rate*num_bottle_layers
        self.trans_1 = TransitionBlock(in_channels=in_trans_1, theta=theta)

        self.dense_2 = DenseBlock(in_channels=int(in_trans_1*theta), num_layers=num_bottle_layers, growth_rate=growth_rate)
        in_trans_2 = int(in_trans_1*theta) + growth_rate*num_bottle_layers
        self.trans_2 = TransitionBlock(in_channels=in_trans_2, theta=theta)

        self.dense_3 = DenseBlock(in_channels=int(in_trans_2*theta), num_layers=num_bottle_layers, growth_rate=growth_rate)
        
        in_fc = int(in_trans_2*theta) + growth_rate*num_bottle_layers
        self.fc = torch.nn.Linear(in_fc, 10)


    def forward(self, x):
        out_init = self.conv(x)
        
        print(out_init.shape)
        out_dense_1 = self.dense_1(out_init)
        out_trans_1 = self.trans_1(out_dense_1)

        out_dense_2 = self.dense_2(out_trans_1)
        out_trans_2 = self.trans_2(out_dense_2)

        out_dense_3 = self.dense_3(out_trans_2)

        gap = torch._adaptive_avg_pool2d(out_dense_3)
        flatten = torch.flatten(gap)

        return flatten


class DenseBlock(torch.nn.Sequential):
    def __init__(self, in_channels, num_layers, growth_rate):
        super().__init__()
        
        for i in range(num_layers):
            in_ch = in_channels + growth_rate * i
            self.add_module('bottleneck', BottleNeck(in_ch, growth_rate))


class BottleNeck(torch.nn.Sequential):
    def __init__(self, in_channels, growth_rate):
        super().__init__()

        self.add_module('conv_1x1', ConvBlock(in_channels, growth_rate*4, kernel=1, stride=1, padding=0))
        self.add_module('conv_3x3', ConvBlock(growth_rate*4, growth_rate, kernel=3, stride=1, padding=1))

    def forward(self, x):
        out = super().forward(x)
        out = torch.concat([x, out], dim=1)
        return out


class TransitionBlock(torch.nn.Sequential):
    def __init__(self, in_channels, theta=0.5):
        super().__init__()
        self.add_module('conv_1x1', ConvBlock(in_channels, int(in_channels*theta), kernel=1, stride=1, padding=0))
        self.add_module('avg_pool', torch.nn.AvgPool2d(2))


class ConvBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel, stride, padding):
        super().__init__()
        self.bn = torch.nn.BatchNorm2d(in_channels)
        self.relu = torch.nn.ReLU(True)
        self.conv = torch.nn.Conv2d(in_channels, out_channels, kernel_size=kernel, stride=stride, padding=padding, bias=False)

    def forward(self, x):
        x = self.bn(x)
        x = self.relu(x)
        x = self.conv(x)
        return x




In [2]:
from torch import nn

class bn_relu_conv(nn.Module):
    def __init__(self, nin, nout, kernel_size, stride, padding, bias=False):
        super(bn_relu_conv, self).__init__()
        self.batch_norm = nn.BatchNorm2d(nin)
        self.relu = nn.ReLU(True)
        self.conv = nn.Conv2d(nin, nout, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias)

    def forward(self, x):
        out = self.batch_norm(x)
        out = self.relu(out)
        out = self.conv(out)

        return out


class bottleneck_layer(nn.Sequential):
  def __init__(self, nin, growth_rate, drop_rate=0.2):    
      super(bottleneck_layer, self).__init__()
      
      self.add_module('conv_1x1', bn_relu_conv(nin=nin, nout=growth_rate*4, kernel_size=1, stride=1, padding=0, bias=False))
      self.add_module('conv_3x3', bn_relu_conv(nin=growth_rate*4, nout=growth_rate, kernel_size=3, stride=1, padding=1, bias=False))
      
      self.drop_rate = drop_rate
      
  def forward(self, x):
      bottleneck_output = super(bottleneck_layer, self).forward(x)
      if self.drop_rate > 0:
          bottleneck_output = torch.dropout(bottleneck_output, p=self.drop_rate, train=self.training)
          
      bottleneck_output = torch.cat((x, bottleneck_output), 1)
      
      return bottleneck_output


class Transition_layer(nn.Sequential):
  def __init__(self, nin, theta=0.5):    
      super(Transition_layer, self).__init__()
      
      self.add_module('conv_1x1', bn_relu_conv(nin=nin, nout=int(nin*theta), kernel_size=1, stride=1, padding=0, bias=False))
      self.add_module('avg_pool_2x2', nn.AvgPool2d(kernel_size=2, stride=2, padding=0))


class DenseBlock(nn.Sequential):
  def __init__(self, nin, num_bottleneck_layers, growth_rate, drop_rate=0.2):
      super(DenseBlock, self).__init__()
                        
      for i in range(num_bottleneck_layers):
          nin_bottleneck_layer = nin + growth_rate * i
          self.add_module('bottleneck_layer_%d' % i, bottleneck_layer(nin=nin_bottleneck_layer, growth_rate=growth_rate, drop_rate=drop_rate))


class DenseNet(nn.Module):
    def __init__(self, growth_rate=12, num_layers=100, theta=0.5, drop_rate=0.2, num_classes=10):
        super(DenseNet, self).__init__()
        
        assert (num_layers - 4) % 6 == 0
        
        # (num_layers-4)//6 
        num_bottleneck_layers = (num_layers - 4) // 6
        
        # 32 x 32 x 3 --> 32 x 32 x (growth_rate*2)
        self.dense_init = nn.Conv2d(3, growth_rate*2, kernel_size=3, stride=1, padding=1, bias=True)
                
        # 32 x 32 x (growth_rate*2) --> 32 x 32 x [(growth_rate*2) + (growth_rate * num_bottleneck_layers)]
        self.dense_block_1 = DenseBlock(nin=growth_rate*2, num_bottleneck_layers=num_bottleneck_layers, growth_rate=growth_rate, drop_rate=drop_rate)

        # 32 x 32 x [(growth_rate*2) + (growth_rate * num_bottleneck_layers)] --> 16 x 16 x [(growth_rate*2) + (growth_rate * num_bottleneck_layers)]*theta
        nin_transition_layer_1 = (growth_rate*2) + (growth_rate * num_bottleneck_layers) 
        self.transition_layer_1 = Transition_layer(nin=nin_transition_layer_1, theta=theta)
        
        # 16 x 16 x nin_transition_layer_1*theta --> 16 x 16 x [nin_transition_layer_1*theta + (growth_rate * num_bottleneck_layers)]
        self.dense_block_2 = DenseBlock(nin=int(nin_transition_layer_1*theta), num_bottleneck_layers=num_bottleneck_layers, growth_rate=growth_rate, drop_rate=drop_rate)

        # 16 x 16 x [nin_transition_layer_1*theta + (growth_rate * num_bottleneck_layers)] --> 8 x 8 x [nin_transition_layer_1*theta + (growth_rate * num_bottleneck_layers)]*theta
        nin_transition_layer_2 = int(nin_transition_layer_1*theta) + (growth_rate * num_bottleneck_layers) 
        self.transition_layer_2 = Transition_layer(nin=nin_transition_layer_2, theta=theta)
        
        # 8 x 8 x nin_transition_layer_2*theta --> 8 x 8 x [nin_transition_layer_2*theta + (growth_rate * num_bottleneck_layers)]
        self.dense_block_3 = DenseBlock(nin=int(nin_transition_layer_2*theta), num_bottleneck_layers=num_bottleneck_layers, growth_rate=growth_rate, drop_rate=drop_rate)
        
        nin_fc_layer = int(nin_transition_layer_2*theta) + (growth_rate * num_bottleneck_layers) 
        
        # [nin_transition_layer_2*theta + (growth_rate * num_bottleneck_layers)] --> num_classes
        self.fc_layer = nn.Linear(nin_fc_layer, num_classes)
        
    def forward(self, x):
        dense_init_output = self.dense_init(x)
        
        dense_block_1_output = self.dense_block_1(dense_init_output)
        transition_layer_1_output = self.transition_layer_1(dense_block_1_output)
        
        dense_block_2_output = self.dense_block_2(transition_layer_1_output)
        transition_layer_2_output = self.transition_layer_2(dense_block_2_output)
        
        dense_block_3_output = self.dense_block_3(transition_layer_2_output)
        
        global_avg_pool_output = torch._adaptive_avg_pool2d(dense_block_3_output, (1, 1))                
        global_avg_pool_output_flat = global_avg_pool_output.view(global_avg_pool_output.size(0), -1)

        output = self.fc_layer(global_avg_pool_output_flat)
        
        return output


In [3]:
# GAP
# noramlize
dummy = torch.rand(size=(4, 3, 224, 224), dtype=torch.float32)
model = DenseNet()
model(dummy).shape

torch.Size([4, 10])

In [38]:
conv = torch.nn.Conv2d(3, 4*2, kernel_size=3, stride=3, padding=10, bias=True)
conv(dummy).shape


torch.Size([4, 8, 81, 81])

In [7]:
type(torch.dropout())

TypeError: dropout() missing 3 required positional argument: "input", "p", "train"